In [1]:
import glob
import cv2
import numpy as np
import random
import matplotlib.image as mpimg

np.set_printoptions(precision=3, suppress=True)

def weighted_img(img, initial_img):
    alpha=0.8 
    beta=1 
    gimel=0.
    return cv2.addWeighted(initial_img, alpha, img, beta, gimel)

def draw_fitline(img, result_l,result_r, color=(255,0,0), thickness = 10):
    # draw fitting line
    lane = np.zeros_like(img)
    
    if result_l == result_r :
        print("same")
        print("Lane is Changing")
        cv2.line(lane, (int(result_l[0]) , int(result_l[1])), (int(vanishing_point[0]), int(vanishing_point[1])), color, thickness)
        cv2.line(lane, (int(result_r[0]) , int(result_r[1])), (int(vanishing_point[0]), int(vanishing_point[1])), color, thickness)
        #cv2.line(lane, (int(result_l[0]) , int(result_l[1])), (int(result_l[2]), int(result_l[3])), color, thickness)
        #cv2.line(lane, (int(result_r[0]) , int(result_r[1])), (int(result_r[2]), int(result_r[3])), color, thickness)
    else :
        print("not same")
        vanishing_line = [0,vanishing_point[1],img.shape[1], vanishing_point[1]]
        a = np.array([result_l, result_r, vanishing_line])
        b = np.array(lineIntersection(a), np.int32)
        p1 = b[np.where(b[:] == vanishing_point[1])[0][0]]
        p2 = b[np.where(b[:] == vanishing_point[1])[0][1]]
        
        if p1[0] > p2[0] :
            tmp = p1
            p1 = p2
            p2 = tmp
            
        if result_l[0] > result_r[0] :
            tmp = result_l
            result_l = result_r
            result_r = tmp
    
        lane_range = np.array([[(result_l[0], result_l[1]),
                                 (p1[0], p1[1]),
                                 (p2[0], p2[1]),
                                 (result_r[0], result_l[1])]], np.int32)
        
        cv2.fillPoly(lane, lane_range, color)
    # add original image & extracted lane lines
    #cv2.line(lane, (int(result_l[0]) , int(result_l[1])), (int(result_l[2]), int(result_l[3])), color, thickness)
    #cv2.line(lane, (int(result_r[0]) , int(result_r[1])), (int(result_r[2]), int(result_r[3])), color, thickness)
    final = weighted_img(lane, img)  
    return final

def grayscale(img) :
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img):
    kernel_size = 5
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img):
    low_threshold = 50
    high_threshold = 200
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img):
    mask = np.zeros_like(img)
    vertices = np.array([[(0, img.shape[0]),
                          ((1/2)*img.shape[1],(1/5)*img.shape[0]),
                          (img.shape[1], img.shape[0])]], np.int32)
    #vertices = np.array([[(0, img.shape[0]),
    #                      ((1/5)*img.shape[1],(1/2)*img.shape[0]),
    #                      ((3/7)*img.shape[1],(1/3)*img.shape[0]),
    #                      ((4/7)*img.shape[1],(1/3)*img.shape[0]),
    #                      ((4/5)*img.shape[1],(1/2)*img.shape[0]),
    #                      (img.shape[1], img.shape[0])]], np.int32)

    ## 기본이미지의 채널이 3이면 mask에 찍을 색상 채널을 3으로 맞추기
    if len(img.shape) >  2 :
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)         ## mask: 색을 찍을 이미지, vertices: 다각형 위치 좌표, ignore:색상
    
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img) :
    rho = 2
    theta = np.pi/180
    threshold = 90
    min_line_len = 120
    max_line_gap = 150
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    #draw_lines(line_img, lines)
    
    return lines

def ToHomogeneous(v):
    return np.hstack((v, 1))

def lineIntersection(lines):
    point = []
    for i in range(0,len(lines)) :
        for j in range(i+1, len(lines)):
            p1h = ToHomogeneous(lines[i,0:2])
            p2h = ToHomogeneous(lines[i,2:])
            p3h = ToHomogeneous(lines[j,0:2])
            p4h = ToHomogeneous(lines[j,2:])
            
            l1 = np.cross(p1h, p2h)
            l2 = np.cross(p3h, p4h)
            l3 = np.cross(l1, l2)
            
            det = l3[-1]                         # 교점이 존재하지 않거나 p1, p2, p3, p4 가 동일 직선 위에 놓인 경우 0
            if det != 0 :
                l3 = l3/det
            
            point.append(l3[:2])
    
    return point

def find_v_point(intersection_p) :
    p = np.array(intersection_p, np.int32).tolist()
    vote = np.zeros((img.shape[0], img.shape[1]), np.int32)
    for i in range(0, len(p)) :
        if p[i][0] < vote.shape[1] and p[i][1] < vote.shape[0] and p[i][0] > 0 and p[i][1] > 0 :
            vote[p[i][1],p[i][0]] += 1
            
    max_cnt = 0
    max_idx = []
    for i in range(0, vote.shape[0]-100+1):
        for j in range(0, vote.shape[1]-100+1):
            temp = vote[i:i+100,j:j+100]
            if max_cnt < sum(temp[temp>0]) :
                max_cnt = sum(temp[temp>0])
                max_idx = [i,j]
                
    r1 = max_idx[0]
    c1 = max_idx[1]
    temp = vote[r1:r1+100, c1:c1+100]
    r2,c2 = np.where(temp == temp.max())[0:2]
    
    if len(r2) > 1 :
        r2 = r2[0]
        c2 = c2[0]
        
    vanishing_point = [c1+int(c2), r1+int(r2)]
    print(vanishing_point, max_cnt)
    return vanishing_point

def get_fitline(img, f_lines):
    rows,cols = img.shape[:2]
    output = cv2.fitLine(f_lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx + x) , int(img.shape[0]/2+100)
    result = [x1,y1,x2,y2]

    return result

def compute_distance(par, point):
    # distance between line & point
    return np.abs(par[0]*point[:,0]+par[1]*point[:,1]+par[2])/np.sqrt(par[0]**2+par[1]**2)

def erase_outliers(par, lines):
    # distance between best line and sample points
    distance = compute_distance(par,lines)

    #filtered_dist = distance[distance<15]
    filtered_lines = lines[distance<13,:]
    return filtered_lines

def model_verification(par, lines):
    # calculate distance
    distance = compute_distance(par,lines)
    # total sum of distance between random line and sample points    
    sum_dist = distance.sum(axis=0)
    # average
    avg_dist = sum_dist/len(lines)
    
    return avg_dist

def compute_model_parameter(line):
    # y = mx+n
    m = (line[3] - line[1])/(line[2] - line[0])
    n = line[1] - m*line[0]
    # ax+by+c = 0
    a, b, c = m, -1, n
    par = np.array([a,b,c])
    return par

def get_random_samples(lines):
    one = random.choice(lines)
    two = random.choice(lines)
    if(two[0]==one[0]): # extract again if values are overlapped
        while two[0]==one[0]:
            two = random.choice(lines)
    one, two = one.reshape(1,2), two.reshape(1,2)
    three = np.concatenate((one,two),axis=1)
    three = three.squeeze()
    return three

def ransac_line_fitting(img, lines, min=100):
    global fit_result, l_fit_result, r_fit_result
    best_line = np.array([0,0,0])
    if(len(lines)!=0):                
        for i in range(100):           
            sample = get_random_samples(lines)
            parameter = compute_model_parameter(sample)
            cost = model_verification(parameter, lines)                        
            if cost < min: # update best_line
                min = cost
                best_line = parameter
            if min < 3: break
        # erase outliers based on best line
        filtered_lines = erase_outliers(best_line, lines)
        fit_result = get_fitline(img, filtered_lines)
    else:
        if (fit_result[3]-fit_result[1])/(fit_result[2]-fit_result[0]) < 0:
            l_fit_result = fit_result
            return l_fit_result
        else:
            r_fit_result = fit_result
            return r_fit_result

    if (fit_result[3]-fit_result[1])/(fit_result[2]-fit_result[0]) < 0:
        l_fit_result = fit_result
        return l_fit_result
    else:
        r_fit_result = fit_result
        return r_fit_result
    
def Collect_points(lines):
    interp = lines.reshape(lines.shape[0]*2,2)
    
    for line in lines:
        if np.abs(line[3]-line[1]) > 5:        ## y축의 변화량이 5이상인 것들만, 즉 화면상 가로로 되어 있는 선은 취급안한다.
            tmp = np.abs(line[3]-line[1])
            a = line[0] ; b = line[1] ; c = line[2] ; d = line[3]
            slope = (line[2]-line[0])/(line[3]-line[1]) 
            for m in range(0,tmp+1,5):
                if slope>0:
                    new_point = np.array([[int(a+m*slope),int(b+m)]])
                    interp = np.concatenate((interp,new_point),axis = 0)
                elif slope<0:
                    new_point = np.array([[int(a-m*slope),int(b-m)]])
                    interp = np.concatenate((interp,new_point),axis = 0)  
    return interp

def detect_lane_img(img) :
    gray = grayscale(img)
    blur_gray =  gaussian_blur(gray)
    edges = canny(blur_gray)
    mask = region_of_interest(edges)
    lines = np.squeeze(hough_lines(mask))
    
    global vanishing_point
    intersection_p = lineIntersection(lines)
    vanishing_point = find_v_point(intersection_p)
    if len(lines.shape) == 1 :
        lines = lines.reshape(1,-1)
        
    slope_degree = (np.arctan2(lines[:,1] - lines[:,3], lines[:,0] - lines[:,2]) * 180) / np.pi
    
    # 수평 기울기 제한
    lines = lines[np.abs(slope_degree)<160]
    slope_degree = slope_degree[np.abs(slope_degree)<160]
    
    # 수직 기울기 제한
    lines = lines[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
    
    L_lines, R_lines = lines[(slope_degree>0),:], lines[(slope_degree<0),:]
    
    # interpolation & collecting points for RANSAC
    L_interp = Collect_points(np.concatenate((L_lines.reshape(-1,2), np.array([vanishing_point,]*len(L_lines.reshape(-1,2)))), axis=1))
    R_interp = Collect_points(np.concatenate((R_lines.reshape(-1,2), np.array([vanishing_point,]*len(R_lines.reshape(-1,2))).reshape(-1,2)), axis=1))

    
    # erase outliers based on best line
    left_fit_line = ransac_line_fitting(img, L_interp)
    right_fit_line = ransac_line_fitting(img, R_interp)
    
    final = draw_fitline(img, left_fit_line, right_fit_line)
    
    return final

#if __name__ == "__main__" :
#    path = 'opencv/picture/road/road1/1.jpg'
#    img = mpimg.imread(path)
#    result = detect_lane_img(img)
#    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
#    
#    cv2.imshow('result', result)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
if __name__ == "__main__" :
    images = [mpimg.imread(file) for file in glob.glob("opencv/picture/road/road30/*.jpg")]
    
    for index, img in enumerate(images) :
        result = detect_lane_img(img)
        result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
        result = cv2.resize(result, (640,480))
        
        title_name = 'result' + str(index+1)
        file_path = "opencv/picture/" + title_name + ".jpg"
        
        cv2.imwrite(file_path, result)
        cv2.imshow(title_name,  result)
        
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[688, 248] 288
not same


KeyboardInterrupt: 